# 仕程的需求，口径已经对齐，直接点全部运行

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 归还逾期

In [4]:
# 归还逾期overdue_type=1，2为租赁期间有逾期
sql = '''
select
yat.order_id, yat.add_time, yat.allot_time, yat.agent_user_name,yat.max_day, tprm.search_time, om.status, om.has_actual, tprm.relet_days, tprm.relet_periods,yat.task_number,
yat.应催收金额, yat.实际催回金额
from (select distinct task_number, order_id,add_time,agent_user_name,allot_time, max_day, money_expect 应催收金额, money 实际催回金额 from db_rent.ya_agent_task where overdue_type=1) yat
left join db_digua_business.t_order om on om.id=yat.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=yat.order_id
-- where  yat.max_day IN (26,27,29,31) 
'''
df = query(sql)
# df.drop_duplicates(subset=['order_id'], inplace=True)

In [5]:
t_date = datetime.now().strftime('%Y%m%d%H')
# allot_time任务分配时间
df.loc[:, 'date'] = df.allot_time.dt.strftime('%Y-%m-%d')
df.loc[:, '姓名'] = df.agent_user_name.apply(lambda x: str(x).split('-')[-1])
df.loc[:, '已买断'] = np.where((df.status==8)&(df.has_actual==1)&(df.relet_days==0), 1, 0)
df.loc[:, '已归还'] = np.where((df.status.isin([8, 15, 16]))&(df.has_actual==0)&(df.relet_days==0), 1, 0)
df.loc[:, '已续租'] = np.where(df.relet_days>0, 1, 0)
df.loc[:, '仍在逾期'] = np.where((df.status==5)&(df.relet_days==0), 1, 0)
df_g = df.groupby('search_time').agg({'order_id': 'count', '已买断': 'sum', '已归还': 'sum', '已续租': 'sum', '仍在逾期': 'sum'}).rename(columns={'order_id': '归还逾期数量'})
df_g = df_g.rename_axis('下单月份')

# # df_g.to_excel('F:/x.xlsx')
# with pd.ExcelWriter(f'F:/需求/仕程需求/归还逾期_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_g.to_excel(writer, sheet_name='归还逾期')

In [6]:
# df_g


## 催收完成情况

In [7]:
# 林大鲲 陈锐 吴威宇 张文奔 郑瑞杰 江文祥 郑文涛 李成铭
df_ghyq = df[(df.date>='2025-07-01')&(df.date<='2025-07-31')&(df.姓名.isin(['林大鲲','吴威宇','陈锐', '张文奔', '郑瑞杰', '江文祥', '郑文涛', '李成铭']))]
# df_ghyq = df_ghyq.sort_values('allot_time').groupby('order_id').head(1)
df_ghyq = df_ghyq.sort_values('allot_time').groupby('task_number').head(1)
df_ghyq_g = df_ghyq.groupby('姓名').agg({'order_id': 'count', '已买断': 'sum', '已归还': 'sum', '已续租': 'sum', '仍在逾期': 'sum', '应催收金额': 'sum', '实际催回金额': 'sum'}).rename(columns={'order_id': '分配数'})
df_ghyq_g.loc[:, '总完成'] = df_ghyq_g.已买断+df_ghyq_g.已归还+df_ghyq_g.已续租
df_ghyq_g.loc[:, '总完成率'] = (df_ghyq_g.总完成/df_ghyq_g.分配数).map(lambda x: format(x, '.2%'))
# df_new_g = df_new_g[['分配数', '总完成', '总完成率']]
# with pd.ExcelWriter('F:/需求/仕程需求/归还逾期催收月末复盘数据_7月.xlsx', engine='xlsxwriter') as writer:
#     df_new_g.to_excel(writer, sheet_name='完成率')
df_ghyq_g

,分配数,已买断,已归还,已续租,仍在逾期,应催收金额,实际催回金额,总完成,总完成率
姓名,,,,,,,,,
吴威宇,76,9,2,26,39,413826.89,70023.93,37,48.68%
张文奔,76,6,4,29,37,395213.41,57275.34,39,51.32%
江文祥,29,5,3,8,13,167209.56,46040.55,16,55.17%
郑瑞杰,76,10,3,29,33,393914.06,69866.62,42,55.26%
陈锐,76,9,5,29,31,420192.13,81535.36,43,56.58%


# 租金逾期

In [8]:
# tprm.overdue_type,yat.催收状态,3为已完成其它都为逾期
# yat.max_day IN (26,27,29,31) max_day为数据类型，仅需要这4个类型
sql_zj = '''
select
yat.order_id, yat.add_time, yat.allot_time, yat.催收状态, yat.agent_user_name, yat.max_day, tprm.search_time, om.status, om.has_actual, tprm.relet_days, tprm.relet_periods
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2
,ymos.refund_date, ymos.reality_refund_date, yat.应催收金额, yat.实际催回金额,yat.task_number, yat.agent_user_id
from (select distinct task_number,order_id,add_time,agent_user_name,allot_time,status 催收状态, max_day, money_expect 应催收金额, money 实际催回金额, agent_user_id from db_rent.ya_agent_task where overdue_type=0) yat
left join db_digua_business.t_order om on om.id=yat.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=yat.order_id
left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
where date_format(add_time,'%Y-%m-%d')>='2025-07-01'
and date_format(add_time,'%Y-%m-%d')<='2025-07-31'
and  yat.max_day IN (26,27,29,31) 
'''
df_zj = query(sql_zj)

In [9]:

# df_123 = df_12[df_12.agent_user_name=='自有催收组-陈锐']
# # 检查order_id是否存在重复    检测     
# df_123[df_123.task_number.duplicated()]
# df_new.shape
# df_zj[df_zj.agent_user_id==420].shape

In [13]:
t_date = datetime.now().strftime('%Y%m%d%H')
name_list = ['自有催收组-吴威宇','自有催收组-陈锐', '自有催收组-张文奔', '自有催收组-郑瑞杰','自有催收组-林大鲲','自有催收组-郑文涛', '自有催收组-李成铭', '自有催收组-江文祥','自有催收组-江文详']# ,'自有催收组-江文祥'
# 筛选agent_user_name包含深圳市润广贸易有限公司和在name_list中的人
# 使用na=False参数处理NaN值
df_zjyq = df_zj[
    df_zj.agent_user_name.str.contains('深圳市润广贸易有限公司', na=False) | 
    df_zj.agent_user_name.isin(name_list) 
    # | (df_zj.agent_user_id == 420)
]
# allot_time任务分配时间    2025-07-01
# df_zjyq.drop_duplicates(subset=['order_id'], inplace=True)      
df_zjyq.loc[:, 'date'] = df_zjyq.allot_time.dt.strftime('%Y-%m-%d')
df_zjyq.loc[:, '姓名'] = df_zjyq.agent_user_name
# df_zjyq.loc[:, '姓名'] = np.where(df_zjyq.agent_user_id==420, '自有催收组-江文祥', df_zjyq.agent_user_name)
df_zjyq.loc[:, '已买断'] = np.where((df_zjyq.status.isin([3, 4]))&(df_zjyq.has_actual==1), 1, 0)
df_zjyq.loc[:, '已还款'] = np.where((df_zjyq.催收状态.isin([3, 4])), 1, 0)
# df_zjyq.loc[:, '已续租'] = np.where(df_zjyq.relet_days>0, 1, 0)
df_zjyq.loc[:, '仍在逾期'] = np.where(~df_zjyq.催收状态.isin([3, 4]), 1, 0)


# df_zjyq = df[(df.date>='2025-07-01')&(df.date<='2025-07-31')&(df.姓名.isin(name_list))]
# df_zjyq.drop_duplicates(subset=['task_number'], inplace=True)
# 取最新任务分配时间的任务号记录，不按订单号是因为可能一个订单号有多个任务号，比如租金催收和归还催收等等
df_zjyq = df_zjyq.sort_values('allot_time').groupby('task_number').head(1)
df_zjyq_g = df_zjyq.groupby('姓名').agg({'order_id': 'count', '已买断': 'sum', '已还款': 'sum','仍在逾期': 'sum','应催收金额': 'sum', '实际催回金额': 'sum'}).rename(columns={'order_id': '分配数'})
df_zjyq_g.loc[:, '总完成'] = df_zjyq_g.已买断+df_zjyq_g.已还款
df_zjyq_g.loc[:, '总完成率'] = (df_zjyq_g.总完成/df_zjyq_g.分配数).map(lambda x: format(x, '.2%'))

with pd.ExcelWriter('F:/需求/仕程需求/催收月末复盘数据_7月.xlsx', engine='xlsxwriter') as writer:
    df_zjyq_g.to_excel(writer, sheet_name='租金逾期完成情况')
    df_ghyq_g.to_excel(writer, sheet_name='归还逾期完成情况')
    
df_zjyq_g

,分配数,已买断,已还款,仍在逾期,应催收金额,实际催回金额,总完成,总完成率
姓名,,,,,,,,
深圳市润广贸易有限公司-chenzemei,190,0,102,88,113310.62,60271.94,102,53.68%
深圳市润广贸易有限公司-mocaidie,205,0,114,91,122669.73,67235.60,114,55.61%
深圳市润广贸易有限公司-wucheng,197,0,113,84,141850.93,81192.66,113,57.36%
深圳市润广贸易有限公司-wuwen003,158,0,90,68,103373.21,59032.77,90,56.96%
深圳市润广贸易有限公司-ying,159,0,90,69,100640.85,55615.22,90,56.60%
深圳市润广贸易有限公司-zhuxiaojuan,151,0,84,67,94486.58,53190.55,84,55.63%
自有催收组-吴威宇,565,0,354,211,362969.45,228418.82,354,62.65%
自有催收组-张文奔,562,0,378,184,359409.90,239374.51,378,67.26%
自有催收组-江文祥,382,0,250,132,247036.92,160847.19,250,65.45%


In [121]:
df_ghyq_g

,分配数,已买断,已归还,已续租,仍在逾期,应催收金额,实际催回金额,总完成,总完成率
姓名,,,,,,,,,
吴威宇,76,9,2,26,39,413826.89,70023.93,37,48.68%
张文奔,76,6,4,29,37,395213.41,57275.34,39,51.32%
江文祥,29,5,3,8,13,167209.56,46040.55,16,55.17%
郑瑞杰,76,10,3,29,33,393914.06,69866.62,42,55.26%
陈锐,76,9,5,29,31,420192.13,81535.36,43,56.58%


In [119]:
sql2 = '''
select
yat.order_id, yat.add_time, yat.sort, tprm.search_time, om.status, om.has_actual, tprm.relet_days, tprm.relet_periods
from (select order_id, add_time, sort from db_rent.ya_agent_task where overdue_type=0) yat
left join db_digua_business.t_order om on om.id=yat.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=yat.order_id
'''
df2 = query(sql2)
df2.drop_duplicates(subset=['order_id'], inplace=True)

In [120]:
df2.loc[:, 'add_date'] = pd.to_datetime(df2.add_time.dt.date)
df2_new = df2[(df2.add_date>=pd.to_datetime('2024-10-01'))&(df2.add_date<=pd.to_datetime('2025-03-31'))]
df2_new

,order_id,add_time,sort,search_time,status,has_actual,relet_days,relet_periods,add_date
21278,351928,2024-10-01 00:30:02,[17],2023-05,8,1,184.0,7.0,2024-10-01
21280,424930,2024-10-01 00:30:02,[16],2023-06,8,1,360.0,12.0,2024-10-01
21282,425376,2024-10-01 00:30:02,[16],2023-06,4,0,122.0,5.0,2024-10-01
21283,425924,2024-10-01 00:30:02,[16],2023-06,8,1,332.0,12.0,2024-10-01
21286,500879,2024-10-01 00:30:02,[15],2023-07,4,0,62.0,3.0,2024-10-01
...,...,...,...,...,...,...,...,...,...
58990,1926032,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58991,1926180,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58992,1926626,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58995,2048920,2025-03-31 00:30:04,[3],2025-01,4,0,0.0,0.0,2025-03-31
